# Model based collaborative system

[Top N-primeiros](https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-get-the-top-n-recommendations-for-each-user)

In [14]:
# import core modules
import pandas as pd 
import numpy as np
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise.accuracy import rmse, mae
# import warnings
# warnings.filterwarnings('ignore')

In [15]:
# rating dataset
ratings = pd.read_csv ('data_movie/ratings.csv')

# movies dataset
movies = pd.read_csv('data_movie/movies.csv')

In [16]:
dict_movies = {}
for index, row in movies.iterrows():
    dict_movies[row['movieId']] = (row['title'], row['genres'])

In [17]:
# adding corresponded data columns
new_cols = np.zeros((ratings.shape[0] , 2)).astype('object')

for index, row in ratings.iterrows():
    new_cols[index][0] = dict_movies[int(row['movieId'])][0]
    new_cols[index][1] = dict_movies[int(row['movieId'])][1]

In [18]:
# final dataframe
aux = np.append(ratings.to_numpy(),new_cols, axis=1)
movielens_dataframe = pd.DataFrame(aux, columns = ['userId', 'movieId', 'rating', 'timestamp','title', 'genres'])
movielens_dataframe.drop(['timestamp'], axis=1, inplace=True)

In [21]:
movielens_dataframe['userId'] = movielens_dataframe['userId'].astype('int32')
movielens_dataframe['movieId'] = movielens_dataframe['movieId'].astype('int64')
movielens_dataframe['rating'] = movielens_dataframe['rating'].astype('int16')

In [33]:
# matriz de ratings-users
movie_matrix = movielens_dataframe.pivot_table(index='userId', columns='title', values='rating')

In [ ]:
# surprise dataset
reader = Reader()
dataset = Dataset.load_from_df(movielens_dataframe[['userId', 'movieId', 'rating']], reader)

In [ ]:
# train set
train_set = dataset.build_full_trainset()

In [ ]:
# svd model
svd = SVD()
svd.fit(train_set)

In [ ]:
# test set
test_set = train_set.build_anti_testset()

In [ ]:
# predictions
pred = svd.test(test_set)

In [ ]:
# metrics
results = (rmse(pred),mae(pred))